# MiniProject_1

In [30]:
import emotion as emo
import nltk
import MPTs as MT
import os
import glob
os.chdir("news_articles")
folders = glob.glob("*")

In [31]:
def fileopener(foldernames) :
    articles = {}
    dates = {}
    txt3 = ''
    for x in foldernames : #폴더 열기
        os.chdir(x)
        files = glob.glob("*.txt")
        for f in files : #파일열기
            f = open(f, "r")
            txt = f.read()
            f.close() #파일열기 끝
            txt1 = txt.split("\n")
            while '' in txt1 :
                txt1.remove('')
            #txt2.extend(txt1) #제목과 내용 분리를 위한 사전작업
            for s in txt1[2:] :
                txt3 += s
            articles[txt1[1]] = txt3
            #txt2 = []
            txt3 = ''
        dates[x] = articles #딕셔너리안에 딕셔너리 작업
        articles = {}
        os.chdir("..")
    return dates

In [32]:
file = fileopener(folders)

In [33]:
file["fox161107"].keys()

dict_keys(['The year of the Latinos: Hispanic voting surge shaking up presidential election', 'Long lines, technical glitches greet some voters across the country', 'LA county elections boss criticized amid voter fraud claims', 'Meet Darren Soto, poised to make history as first Florida Puerto Rican in Congress', 'Latinos still far from the White House but could make history on down-ballot races this year', 'The Halftime Report endorsementThe polling place on the North Fork of Short Creek in Ohio County, W.Va. was a fine place to cast a ballot. It was a 100-year-old Methodist church with groaning floorboards that announced your arrival for the poll workers before you were halfway across the floor.', "What to watch for in Tuesday's tight race for Senate control", "'Sleeping Giant' awake and roaring ? early voting shows high Latino turnout", 'In battleground North Carolina, 2016 races too close to call.', "Dying Florida man's family fights to get vote counted", 'Trump, Clinton make final 

In [34]:
for x in file["cnn161107"].keys() :
    print(type(x))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [35]:
file["fox161109"].keys()

dict_keys(['World leaders welcome Trump presidential victory with mixed reactions', 'Canadian immigration website crashes during election', "How two polls predicted Trump's surprise victory", "Trump's presidential victory sparks protests and vigils across the U.S.", 'Trump wins presidency, overriding high Latino turnout fueled by effort to defeat him', "Nevada's Cortez Masto becomes 1st Latina elected to the U.S. Senate", 'Latino leaders see complicated picture in Hispanic election turnout and support', 'Voting glitches, long lines and machine break downs ? but no major problems'])

In [36]:
file.keys()

dict_keys(['cnn161107', 'cnn161109', 'fox161107', 'fox161109'])

# tokenize and prinout

In [37]:
os.chdir("..")

In [38]:
from nltk.tokenize import sent_tokenize, word_tokenize, regexp_tokenize
from nltk.corpus import stopwords # remove words that just contain no meaning
import sentiment_mod as s
pattern= r'''[a-z]*.'''
stop_words = set(stopwords.words('english'))
stop_words.add(".")
stop_words.add(",")
stop_words.add("~!@#$%^&*()_+-=`{}[]|\\:;\"',./<>?-()\"")


#tokens = word_tokenize(string)

#filtered_sentence = [w for w in tokens if not w in stop_words] #이 부분은 무엇인지?

In [39]:
def tokenwswords(sent) :
    sym = "’“”``''``''~!@#$%^&*()_--+-=`{}[]|\\:;\"',./<>?-()\""
    token = word_tokenize(sent)
    ftoken = [w for w in token if not w in stop_words]
    for x in ftoken :
        if x in sym :
            ftoken.remove(x)
    return ftoken

# printout

In [40]:
f = open("output.txt", "w", encoding = "utf-8")

In [41]:
def freqprintout(dict) :
    dia = {}
    dic = {}
    for x in dict.keys() :
        for (k, v) in dict[x].items() :
            for i in tokenwswords(k):
                if i not in dia:
                    dia[i] = 1
                else:
                    dia[i] += 1
            for o in tokenwswords(v):
                if o not in dic:
                    dic[o] = 1
                else:
                    dic[o] += 1
        f = open("output.txt", "a", encoding = "utf-8")
        f.write("word Frequency analysis, company name and the date : " + x + "\n\n")
        f.write("article Freq dict : \n")
        for x in sorted(dia, key=dia.get, reverse=True)[:30]:
            f.write(str(x) + " " +  str(dia[x]) + '\n')
        f.write("\n")
        f.write("content Freq dict : \n")
        for x in sorted(dic, key=dic.get, reverse=True)[:200]:
            f.write(str(x) + " " +  str(dic[x]) + '\n')
        f.write("\n")
        f.close()
        dia = {}
        dic = {}

In [42]:
freqprintout(file)

In [43]:
def sentprintout(dict) :
    for x in dict.keys() :
        f = open("output.txt", "a", encoding = "utf-8")
        f.write("sentimental analysis, company name and the date : " + x + "\n\n")
        for (k, v) in dict[x].items() :
            a_s = s.sentiment(k)
            c_s = s.sentiment(v)
            f.write("article name : " + str(k) + " result : " + str(a_s) + '\n')
            f.write("content analysis result : " + str(c_s) + "\n\n")
        f.close()

In [44]:
sentprintout(file)

In [45]:
def trump_word(dict) : 
    trumptoken = []
    ex = []
    sym = "’“”``''``''~!@#$%^&*()_--+-=`{}[]|\\:;\"',./<>?-()\""
    for (k, v) in dict.items() :
        txta = tokenwswords(k)
        txt = tokenwswords(v)
        for i in range(len(txta)) :
            if txta[i].lower() == "trump" :
                trumptoken.extend(txta[i-4:i+5])
        for i in range(len(txt)) :
            if txt[i].lower() == "trump" :
                trumptoken.extend(txt[i-4:i+5])
    ftoken = [w for w in trumptoken if not w in stop_words]
    for x in ftoken :
        if x in sym :
            ftoken.remove(x)
    for x in ftoken :
        if x.lower() == "trump" :
            ftoken.remove(x)
    return ftoken

In [46]:
def cnn_trump_word_1607(dict) :
    help = []
    name = ["cnn161107"]
    for x in name :
        txt = trump_word(file[x])
        help.extend(txt)
    return help

In [47]:
def cnn_trump_word_1609(dict) :
    help = []
    name = ["cnn161109"]
    for x in name :
        txt = trump_word(file[x])
        help.extend(txt)
    return help

In [48]:
def fox_trump_word_1607(dict) :
    help = []
    name = ["fox161107"]
    for x in name :
        txt = trump_word(file[x])
        help.extend(txt)
    return help

In [49]:
def fox_trump_word_1609(dict) :
    help = []
    name = ["fox161109"]
    for x in name :
        txt = trump_word(file[x])
        help.extend(txt)
    return help

In [50]:
def trump_sent(dict) : 
    trumptoken = []
    ex = []
    end = ".!?"
    sym = "’“”``''``''~!@#$%^&*()_--+-=`{}[]|\\:;\"',./<>?-()\""
    for x in dict.keys() :
        for (k, v) in dict[x].items() :
            if "trump" or "Trump" in k :
                trumptoken.append(k)
            txt = v.split("." or "!" or "?")
            for i in range(len(txt)) :
                if "trump" or "Trump" in txt[i] :
                    trumptoken.append(txt[i])
    return trumptoken

In [51]:
def senttrump(dict) :
    ctw7 = cnn_trump_word_1607(dict)
    ctw9 = cnn_trump_word_1609(dict)
    ftw7 = fox_trump_word_1607(dict)
    ftw9= fox_trump_word_1609(dict)
    txt = ''
    for x in ctw7 :
        txt += x + ' '
    f = open("output.txt", "a", encoding = "utf-8")
    f.write("cnn trump word sentiment analysis 161107: " + str(s.sentiment(txt)) + "\n\n")
    f.close()
    txt = ''
    for x in ctw9 :
        txt += x + ' '
    f = open("output.txt", "a", encoding = "utf-8")
    f.write("cnn trump word sentiment analysis 161109: " + str(s.sentiment(txt)) + "\n\n")
    f.close()
    txt = ''
    for x in ftw7 :
        txt += x + ' '
    f = open("output.txt", "a", encoding = "utf-8")
    f.write("fox trump word sentiment analysis 161107: " + str(s.sentiment(txt)) + "\n\n")
    f.close()
    for x in ftw9 :
        txt += x + ' '
    f = open("output.txt", "a", encoding = "utf-8")
    f.write("fox trump word sentiment analysis 161109: " + str(s.sentiment(txt)) + "\n\n")
    f.close()
    return "done"

In [52]:
senttrump(file)

'done'